In [12]:
import nltk
from nltk.stem.snowball import SnowballStemmer 

import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

stemmer = SnowballStemmer(language='russian')
with open('data/intents.json', encoding="utf8") as f:

    data = json.load(f)

# Initialize lists

In [13]:
try:
    with open('data/data.pickle', "rb") as f:
        words, labels, training, output = pickle.load(f)
    print("initalized from pickle")
except:
    print("initalized from json")
    
    words  = []
    labels = []
    docs_x = []
    docs_y = []

    removed_symbols = ["?", "!"]

    for intent in data["intents"]:

        for pattern in intent['patterns']:

            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(pattern)
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    words = [stemmer.stem(w.lower()) for w in words if w not in removed_symbols]
    words = sorted(list(set(words)))

    labels = sorted(labels)
    
    # creating bag of words
    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):

        bag = []

        wrds = [stemmer.stem(w) for w in doc.split()]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output   = np.array(output)

with open('data/data.pickle', "wb") as f:
    pickle.dump((words, labels, training, output), f)

initalized from json


In [14]:
output.shape

(37, 7)

In [15]:
len(words), len(labels), training.shape, output.shape

(67, 7, (37, 67), (37, 7))

# Train a NN model

In [17]:

try:
    tf.compat.v1.reset_default_graph()
    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
    net = tflearn.regression(net)
    model = tflearn.DNN(net)
    model.load("model.tflearn/model.tflearn")
except Exception as e:
    
    print(f"exception: {e}")
    
    tf.compat.v1.reset_default_graph()

    net = tflearn.input_data(shape=[None, len(training[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
    net = tflearn.regression(net)

    model = tflearn.DNN(net)
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    
    model.save("model.tflearn/model.tflearn")

Training Step: 4999  | total loss: 0.02017 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.02017 - acc: 1.0000 -- iter: 32/37
Training Step: 5000  | total loss: 0.02011 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.02011 - acc: 1.0000 -- iter: 37/37
--
INFO:tensorflow:D:\ONLINE COURSES\Python\NLP\Python Chat Bot Managing New Student Applications\model.tflearn\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
training.shape

(35, 66)

In [7]:
output.shape

(35, 7)

In [18]:
def bag_of_words(s, words):

    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)

In [19]:
arr = bag_of_words('Приветствую', words)
np.array(words)[arr==1]

array(['приветств'], dtype='<U12')

In [20]:
word = 'привет'
probabilities = model.predict([bag_of_words(word, words)])[0]
ind = np.argmax(probabilities)
tag_probability = probabilities[ind]
tag = labels[ind]
tag

'приветствие'

In [12]:
probabilities.shape

(66,)

In [28]:
from Schedule import Schedule
from UserProfile import UserProfile

def ask_user_for_additional_info(user_profile):
    
    # ask user for additional info: first name, last name etc.

    printed_header = False
    for att_name, att_type in user_profile.get_attribute_names(exclude_schedule=True):
        val = user_profile._get_attribute_value(att_name)

        # if attribute value is not filled in
        if not val:

            if not printed_header:
                printed_header = True
                print('Пожалуйста введите дополнительную информацию(для отказа напишите 0):')

            att_name_hf = user_profile.get_attribute_name_hf(att_name)

            if att_type == int:
                while True:
                    val = input(f"{att_name_hf}: ")

                    if val.strip() == '0':
                        break

                    try:
                        int(val)
                        break
                    except:
                        print('Пожалуйста введите число.')

            else:
                val = input(f"{att_name_hf}: ")

            if val.strip() == '0':
                
                continue

            user_profile.update(_att_name=att_name, _att_value=val)

def choose_date(code, user_profile):

    schedule = Schedule()
    schedule.print()
    success = False
    while True:

        numbers = input("Выберите часы в которые вы хотите заниматься(укажите номера через запятую, для выхода наберите 0): ")
        
        if numbers.strip() == '0':
            break

        success, schedule_str = schedule.get_schedule(numbers)
        if success:
            print(f"Выбрано время : {schedule_str.strip('[').strip(']')}")
            user_profile.update(schedule=schedule_str)
            break
        else:
            print('Попробуйте еще раз.')    
    
    if success:
        ask_user_for_additional_info(user_profile)
        user_profile.save()
        if user_profile.contact_info_set():
            return "Заявка оформлена. Мы свяжемся с Вами в ближайшее время."
        else:
            return "Заявка отменена. Отсутствует контактная информация!"
    
    return code

def custom_response(code, user_profile):
    
    if code == "code_schedule_response":
        # show options using buttons
        return choose_date(code, user_profile)
    
    return code

def chat(user_id):
    
    user_profile = UserProfile(user_id)

    print("Чтобы остановить чат напишите `quit`.")
    
    while True:
        
        inp = input("You: ")
        if inp.lower() == 'quit':
            break
            
        probabilities = model.predict([bag_of_words(inp, words)])[0]
        ind = np.argmax(probabilities)
        tag_probability = probabilities[ind]
        tag = labels[ind]

        if tag_probability > 0.7:
            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
            response = custom_response(random.choice(responses), user_profile)
            
            if response:
                print(f"Bot: {response}")
        else:
            
            print("Bot: Я вас не понял. Попробуйте написать по другому.")

print("I am ready for chat")

I am ready for chat


In [29]:
chat(user_id=3)

Чтобы остановить чат напишите `quit`.
You: привет
Bot: Здравствуйте!
You: quit


In [9]:
from UserProfile import UserProfile

In [11]:
user_profile = UserProfile(3)
user_profile.contact_info_set()
user_profile.get_phone()

'987496873456-0'

# Initialize Timetable

In [133]:
import pandas as pd

weekday_names = ['пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс']
times = ['09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00']
df = pd.DataFrame(columns=weekday_names)
for time in times:
    
    df.loc[time] = ''

timetable = [
    ('09:00','пн'),
    ('09:00','вт'),
    ('10:00','пт'),
    ('11:00','пн'),
    ('12:00','пн'),
    ('12:00','сб'),
    ('13:00','вт'),
    ('15:00','ср'),
    ('17:00','пт'),
]

counter = 0
for time, weekday in timetable:
    counter += 1
    df.loc[time, weekday] = counter
    
df.index.name = 'time'

from tabulate import tabulate
print(tabulate(df, headers='keys', tablefmt='grid'))

timetable_path = 'data/schedule.csv'
try:
    df.to_csv(timetable_path)
    print(f'Saved timetable to: {timetable_path}')
except Exception as e:
    print(f"Error: {e}")
    print("Can't save timetable")

+--------+------+------+------+------+------+------+------+
| time   | пн   | вт   | ср   | чт   | пт   | сб   | вс   |
+========+======+======+======+======+======+======+======+
| 09:00  | 1    | 2    |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 10:00  |      |      |      |      | 3    |      |      |
+--------+------+------+------+------+------+------+------+
| 11:00  | 4    |      |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 12:00  | 5    |      |      |      |      | 6    |      |
+--------+------+------+------+------+------+------+------+
| 13:00  |      | 7    |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 14:00  |      |      |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 15:00  |      |      | 8    |      |      |      |      |
+--------+------+------+------+------+--

# class Schedule

In [344]:
# from tabulate import tabulate

class Schedule:
    
    def __init__(self):
        
        self._df = pd.read_csv('data/schedule.csv', index_col = 'time', dtype=object)
        self._df.fillna(0, inplace=True)
        self._df = self._df.astype(int)
    
    def print(self):
        
        print(tabulate(self._df.replace(0,''), headers='keys', tablefmt='grid'))
    
    def get_schedule(self, numbers_str):
        ''' 
        Convert string of numbers to the list of hours from schedule.
        e.g. numbers_str = '3, 5'
        Returns tuple: (success->bool, schedule->str)
        '''
        df = self._df
        schedule = []
        success  = True
        numbers_str = numbers_str.strip(',')
        try:
            numbers = [int(n) for n in numbers_str.split(',')]
            for n in numbers:
                
                if df.eq(n).any().any():
                    # number exists in df
                    # r,c are chosen number's row and column in df
                    r, c = df.index[df.eq(n).any(1)].values[0], df.columns[df.eq(n).any(0)].values[0]
                    schedule.append((r, c))
                else:
                    print(f'Номера {n} нет в таблице.')
                    success = False
                    
        except Exception as e:
                
            if isinstance(e, ValueError):
                print('Неправильный формат.')
                
            success = False

        return success, str(schedule)

# class UserProfile

In [350]:
class UserProfile():
    
    
    def __init__(self, id):
        '''user_profile = {"id":0,      "first_name":"", "last_name":"", 
                           "age":0,     "grade":0,
                           "email":"",  "phone":"",      "schedule":" e.g.[('09:00','пн')]"}
        '''
        
        self._filename = 'data/user_profiles.csv'
        self._df_user_profiles = pd.read_csv(user_profiles_path, index_col='id')
        self._user_profile = {"id":id}

        if id in self._df_user_profiles.index.values:
            # update using data from database
            self._user_profile.update(self._df_user_profiles.loc[id].to_dict())
        else:
            # create empty profile
            self._user_profile.update({col:"" for col in self._df_user_profiles.columns})
            self._user_profile['age'] = self._user_profile['grade'] = 0 # use default values
            self._user_profile['schedule'] = '[]' # use default values
    
    def update(self, first_name="", last_name="", age=0, grade=0,
                     email="", phone="", schedule='[]', _att_name='', _att_value=''):
        
        user_profile = self._user_profile
        
        if first_name != "":
            user_profile["first_name"] = first_name

        if last_name != "":
            user_profile["last_name"] = last_name

        if age != 0:
            user_profile["age"] = age

        if grade != 0:
            user_profile["grade"] = grade

        if email != "":
            user_profile["email"] = email

        if phone != "":
            user_profile["phone"] = phone

        if schedule != '[]':
            user_profile["schedule"] = schedule
            
        if _att_name != "":
            user_profile[_att_name] = _att_value
            
            
    def _get_attribute_value(self, attribute_name):
        
        return self._user_profile[attribute_name]
        
    def get_id(self):
        
        return self._user_profile['id']

    def get_first_name(self):
        
        return self._user_profile['first_name']

    def get_last_name(self):
        
        return self._user_profile['last_name']

    def get_age(self):
        
        return self._user_profile['age']

    def get_grade(self):
        
        return self._user_profile['grade']

    def get_email(self):
        
        return self._user_profile['email']

    def get_phone(self):
        
        return self._user_profile['phone']

    def get_schedule(self):
        
        return self._user_profile['schedule']
    
    def get_attribute_names(self, exclude_schedule=False):
        
        types = {'first_name':str, 'last_name':str, 
                 'age':int, 'grade':int, 
                 'email':str, 'phone':str, 'schedule':str}

        excluded = ['id']
        if exclude_schedule:
            excluded.append('schedule')
            
        return [(key, types[key]) for key in user_profile._user_profile.keys() if key not in excluded]
    
    def get_attribute_name_hf(self, att_name):
        '''get human-friendly attribute name'''
        
        names = {'first_name':'имя', 'last_name':'фамилия', 
                 'age':'возраст', 'grade':'класс', 
                 'email':'email', 'phone':'телефонный номер', 'schedule':'график'}
        
        return names[att_name]
    
    def save(self):
        
        '''save user_profiles to disk'''
        user_profiles = self._df_user_profiles
        user_profile  = self._user_profile
        
        # update user's profile in database table
        user_profiles.loc[user_profile['id']] = pd.Series({k:v for k,v in user_profile.items() if k!='id'})
        user_profiles.to_csv(self._filename)


In [249]:
user_profile = UserProfile(id=0)
user_profile.update(first_name='Гарри', last_name = 'Поттер', age=11, grade=5, email='harry.potter@hogwarts.edu')
user_profile.save()
user_profile = UserProfile(id=1)
user_profile.update(first_name='Рон', last_name = 'Уизли', age=11, grade=5, email='ron.weasley@hogwarts.edu')
user_profile.save()
user_profile = UserProfile(id=2)
user_profile.update(first_name='Гермиона', last_name = 'Грейнджер', age=11, grade=5, email='hermione.gr@hogwarts.edu')
user_profile.save()

In [241]:
user_profile.get_attribute_names()

['first_name', 'last_name', 'age', 'grade', 'email', 'phone', 'schedule']

In [301]:
first_name=""
last_name="" 
age=0
grade=0
email=""
phone=""
schedule=[]

if schedule:
    print(f"value: {schedule}")

# Main

In [354]:
schedule = Schedule()
schedule.print()

user_id = 4 # e.g. telegram user id
user_profile = UserProfile(id=user_id)

success = False
while True:
    
    numbers = input("Выберите часы в которые вы хотите заниматься(укажите номера через запятую, для выхода наберите 0): ")
    
    if numbers.strip() == '0':
        break
        
    success, schedule_str = schedule.get_schedule(numbers)
    if success:
        print(f"Выбрано время : {schedule_str.strip('[').strip(']')}")
        user_profile.update(schedule=schedule_str)
        break
    else:
        print('Попробуйте еще раз.')
        
if success:
    # ask user for additional info: first name, last name etc.
    
    printed_header = False
    for att_name, att_type in user_profile.get_attribute_names(exclude_schedule=True):
        val = user_profile._get_attribute_value(att_name)
        
        # if attribute value is not filled in
        if not val:

            if not printed_header:
                printed_header = True
                print('Пожалуйста введите дополнительную информацию(для отказа напишите 0):')
            
            att_name_hf = user_profile.get_attribute_name_hf(att_name)
            
            if att_type == int:
                while True:
                    val = input(f"{att_name_hf}: ")

                    if val.strip() == '0':
                        break
                        
                    try:
                        int(val)
                        break
                    except:
                        print('Пожалуйста введите число.')
                        
            else:
                val = input(f"{att_name_hf}: ")

            if val.strip() == '0':
                continue

            user_profile.update(_att_name=att_name, _att_value=val)
            
user_profile.save()

+--------+------+------+------+------+------+------+------+
| time   | пн   | вт   | ср   | чт   | пт   | сб   | вс   |
+========+======+======+======+======+======+======+======+
| 09:00  | 1    | 2    |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 10:00  |      |      |      |      | 3    |      |      |
+--------+------+------+------+------+------+------+------+
| 11:00  | 4    |      |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 12:00  | 5    |      |      |      |      | 6    |      |
+--------+------+------+------+------+------+------+------+
| 13:00  |      | 7    |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 14:00  |      |      |      |      |      |      |      |
+--------+------+------+------+------+------+------+------+
| 15:00  |      |      | 8    |      |      |      |      |
+--------+------+------+------+------+--

In [355]:
user_profile._user_profile

{'id': 4,
 'first_name': 'Rus',
 'last_name': 'Mus',
 'age': 0,
 'grade': '3',
 'email': '',
 'phone': '',
 'schedule': "[('09:00', 'пн'), ('09:00', 'вт')]"}

In [357]:
user_profile._df_user_profiles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 4
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  4 non-null      object
 1   last_name   4 non-null      object
 2   age         4 non-null      int64 
 3   grade       4 non-null      object
 4   email       4 non-null      object
 5   phone       4 non-null      object
 6   schedule    4 non-null      object
dtypes: int64(1), object(6)
memory usage: 256.0+ bytes
